In [2]:
from jupyter_plotly_dash import JupyterDash
import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State
import base64
import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import re
from CRUD import AnimalShelter

###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "password"
shelter = AnimalShelter(username, password)

# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))
# Removes ObjectID from dataset
df = df.iloc[:,1:]

PAGE_SIZE=10

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    # Holds the logo and link redirected to when the logo is clicked
    html.A([
        html.Img(
        src='data:image/png;base64,{}'.format(encoded_image.decode()),
        style={
            'height':'5%',
            'width':'5%'            
        })
    ], href='https://www.snhu.edu', target='_blank'),
    # Unique identifier
    html.Center(html.B(html.H1('Cody Calkins SNHU CS-340 Dashboard'))),
    html.Hr(),
    # Holds the filter dropdown
    html.Div([
        dcc.Dropdown(id='filter-id', 
                     options=[
                         {'label':'Water Rescue', 'value':'Water Rescue'},
                         {'label':'Mountain Rescue', 'value':'Mountain Rescue'},
                         {'label':'Disaster Rescue', 'value':'Disaster Rescue'}
                     ],
                     placeholder='Select a Filter...')  
    ], style={'textAlign':'left', 'width':'10%'}),
    html.Hr(),
    # Holds the data table
    dt.DataTable(
        id='datatable-id',
        style_data={'whitespace':'normal', 'height':'auto'},
        data=df.to_dict('records'),
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        editable=False,
        filter_action="native",
        column_selectable=False,
        row_selectable=False,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=PAGE_SIZE
    ),
    html.Br(),
    html.Hr(),
    # This sets up the dashboard so that the pie chart and geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6'
            ),
        html.Div(
            id='map-id',
            className='col s12 m6'
            )
        ])
], style={'textAlign':'center'})

#############################################
# Interaction Between Components / Controller
#############################################    
# Callback for changing query based on selected dropdown filter
@app.callback(
    Output('datatable-id','data'),
    [Input('filter-id', 'value')])
def update_dashboard(value):
        if value == 'Water Rescue':
            df = pd.DataFrame.from_records(shelter.read({"animal_type":"Dog",                                                          
                                                         "breed":re.compile('Labrador|Chesa|Newfound'),
                                                         "sex_upon_outcome":"Intact Female",
                                                         "age_upon_outcome_in_weeks":{"$gt":26, "$lt":156}
                                                        }))
        elif value == 'Mountain Rescue':        
            df = pd.DataFrame.from_records(shelter.read({"animal_type":"Dog", 
                                                         "breed":{"$in":["German Shepherd", "Alaskan Malamute", 
                                                                         "Old English Sheepdog", "Siberian Husky",
                                                                         "Rottweiler"]},
                                                         "sex_upon_outcome":"Intact Male",
                                                         "age_upon_outcome_in_weeks":{"$gt":26, "$lt":156}
                                                        }))
        elif value == 'Disaster Rescue':
            df = pd.DataFrame.from_records(shelter.read({"animal_type":"Dog", 
                                                         "breed":{"$in":["Doberman Pinsch", "German Shepherd", 
                                                                         "Golden Retriever", "Bloodhound", 
                                                                         "Rottweiler"]},
                                                         "sex_upon_outcome":"Intact Male",
                                                         "age_upon_outcome_in_weeks":{"$gt":20, "$lt":300}
                                                        }))
        else:
            df = pd.DataFrame.from_records(shelter.read({}))
      
        df = df.iloc[:,1:]

        return df.to_dict('records')
    
    
# Callback to set data table to page 1 after filter update
@app.callback(
    Output('datatable-id', 'page_current'),
    [Input('filter-id', 'value')])
def page_reset(value):
    return 0

# Callback to set active_cell to None after filter update
@app.callback(
    Output('datatable-id', 'active_cell'),
    [Input('filter-id', 'value')])
def cell_reset(value):
    return None
    
# Callback for populating pie chart with shown data
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    return [
        dcc.Graph(            
            figure = px.pie(dff, names='breed')
        )    
    ]

# Creates map and places marker at selected entries' location
@app.callback(
    Output('map-id', "children"),
    [
     Input('datatable-id', 'derived_viewport_data'),
     Input('datatable-id', 'active_cell')
    ]    
)
def update_map(viewData, active_cell):
    dff = pd.DataFrame.from_dict(viewData)
    
    # If a cell is selected, get row and set lat, lon, breed, and name to associated values
    # Otherwise, use values for first row on page
    if active_cell:
        active_row = active_cell['row']
        lat = dff.iloc[active_row,13]
        lon = dff.iloc[active_row,14]
        breed = dff.iloc[active_row,4]
        name = dff.iloc[active_row,9]
    else:
        active_row_id = None
        lat = dff.iloc[0,13]
        lon = dff.iloc[0,14]
        breed = dff.iloc[0,4]
        name = dff.iloc[0,9]
    
    return[
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[lat, lon], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[lat, lon], children=[
                dl.Tooltip(breed),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(name)
                ])
            ])
        ])
    ]


app